# Worksheet 08

Name:  John Salloum
UID: U13233738

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [34]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.2241030546858696, 4.258823235883537, 5.377695337613099, 6.70013444526504, 6.423830755628545, 6.273082570891855, 4.827992244456105, 6.183300354484212, 5.878435488341024, 5.239692643946462]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [35]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[9.013780652608993, 7.858575115710112, 9.483948631331907, 7.967749958701342, 8.406982644157026, 7.609372672427472, 6.784287923673, 7.328738452888659, 7.447935929995817, 9.27319682494784]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [36]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[9.27319682494784, 7.447935929995817, 5.239692643946462, 5.878435488341024, 6.183300354484212, 4.827992244456105, 6.273082570891855, 7.328738452888659, 6.423830755628545, 6.70013444526504]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Probability of being species 1, species 2, mean and variance of weight of species 1, mean and varience of weight of species 2

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [37]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) ,  sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[9.27319682494784, 7.447935929995817, 7.328738452888659]
[5.239692643946462, 5.878435488341024, 6.183300354484212, 4.827992244456105, 6.273082570891855, 6.423830755628545, 6.70013444526504]
P(S_1) = 0.3,  P(S_2) = 0.7
mean_1 = 8.016623735944107,  mean_2 = 5.932352643287607
var_1 = 0.7918559704289789,  var_2 = 0.3874868703528599


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [38]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    #pdf_i[0] = p(p | s_0)

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]* prob_s[0]/prob_x )
    prob_s1_x.append( pdf_i[1]* prob_s[1]/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  9.27319682494784
probability of observing that point if it came from cluster 0 =  0.1430383954284032
probability of observing that point if it came from cluster 1 =  7.427162158513885e-17
point =  7.447935929995817
probability of observing that point if it came from cluster 0 =  0.38928350510262494
probability of observing that point if it came from cluster 1 =  0.0004905131227923362
point =  5.239692643946462
probability of observing that point if it came from cluster 0 =  0.0010758869679249655
probability of observing that point if it came from cluster 1 =  0.20834312731570614
point =  5.878435488341024
probability of observing that point if it came from cluster 0 =  0.013152193181624897
probability of observing that point if it came from cluster 1 =  1.019644471617352
point =  6.183300354484212
probability of observing that point if it came from cluster 0 =  0.034538100405361885
probability of observing that point if it came from cluster 1 =  0.8347885317557792
point =  4.8

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [39]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.3,  P(S_2) = 0.7
mean_1 = 7.853260170554176,  mean_2 = 5.898149600704986
var_1 = 0.9247173108934132,  var_2 = 0.384362776431258


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)